<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C2/blob/master/W3/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Language Models: Auto-Complete
In this assignment, you will build an auto-complete system. Auto-complete system is something you may see every day

When you google something, you often have suggestions to help you complete your search.
When you are writing an email, you get suggestions telling you possible endings to your sentence.
By the end of this assignment, you will develop a prototype of such a system.

A key building block for an auto-complete system is a language model. A language model assigns the probability to a sequence of words, in a way that more "likely" sequences receive higher scores. For example,

* "I have a pen" is expected to have a higher probability than "I am a pen" since the first one seems to be a more natural sentence in the real world.

You can take advantage of this probability calculation to develop an auto-complete system.
Suppose the user typed

* "I eat scrambled" Then you can find a word x such that "I eat scrambled x" receives the highest probability. If x = "eggs", the sentence would be "I eat scrambled eggs"

While a variety of language models have been developed, this assignment uses N-grams, a simple but powerful method for language modeling.

* N-grams are also used in machine translation and speech recognition.
Here are the steps of this assignment:

1. Load and preprocess data
2. Load and tokenize data.
3. Split the sentences into train and test sets.
4. Replace words with a low frequency by an unknown marker <unk>.
5. Develop N-gram based language models
  * Compute the count of n-grams from a given data set.
  * Estimate the conditional probability of a next word with k-smoothing.
6. Evaluate the N-gram models by computing the perplexity score.
7. Use your own model to suggest an upcoming word given your sentence.

In [1]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.data.path.append('.')

!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
--2020-10-31 14:09:40--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C2/master/W3/en_US.twitter.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3341555 (3.2M) [text/plain]
Saving to: ‘en_US.twitter.txt’

en_US.twitter.txt   100%[===================>]   3.19M  --.-KB/s    in 0.1s    

2020-10-31 14:09:41 (29.3 MB/s) - ‘en_US.twitter.txt’ saved [3341555/3341555]



In [2]:
with open("en_US.twitter.txt", "r") as f:
    data = f.read()
print("Data type:", type(data))
print("Number of letters:", len(data))
print("First 300 letters of the data")
print("-------")
display(data[0:300])
print("-------")

print("Last 300 letters of the data")
print("-------")
display(data[-300:])
print("-------")

Data type: <class 'str'>
Number of letters: 3335477
First 300 letters of the data
-------


"How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way too long.\nWhen you meet someone special... you'll know. Your heart will beat more rapidly and you'll smile for no reason.\nthey've decided its more fun if I don't.\nSo Tired D; Played Lazer Tag & Ran A "

-------
Last 300 letters of the data
-------


"ust had one a few weeks back....hopefully we will be back soon! wish you the best yo\nColombia is with an 'o'...“: We now ship to 4 countries in South America (fist pump). Please welcome Columbia to the Stunner Family”\n#GutsiestMovesYouCanMake Giving a cat a bath.\nCoffee after 5 was a TERRIBLE idea.\n"

-------


## Part 1.2 Pre-process the data
Preprocess this data with the following steps:

1. Split data into sentences using "\n" as the delimiter.
2. Split each sentence into tokens. Note that in this assignment we use "token" and "words" interchangeably.
3. Assign sentences into train or test sets.
4. Find tokens that appear at least N times in the training data.
5. Replace tokens that appear less than N times by <unk>

**Note: we omit validation data in this exercise.**

In real applications, we should hold a part of data as a validation set and use it to tune our training.
We skip this process for simplicity.

## Exercise 01
Split data into sentences.

In [3]:
def split_to_sentences(data):
    """
    Take the full corpus and split into sentences.
    Input:
    data: a str that I read from the file
    Output:
    List of sentences    
    """
    sent = data.split("\n")

    # Remove leading and trailing spaces

    sent = [s.strip() for s in sent]

    # Remove empty sentences

    sent = [s for s in sent if len(s) > 0]

    return sent

In [4]:
# Test

X = """
I have a pen.\nI have an apple. \nAh\nApple pen.\n
"""

y = split_to_sentences(X)

print(y)

['I have a pen.', 'I have an apple.', 'Ah', 'Apple pen.']


## Exercise 02
The next step is to tokenize sentences (split a sentence into a list of words).

* Convert all tokens into lower case so that words which are capitalized (for example, at the start of a sentence) in the original text are treated the same as the lowercase versions of the words.
* Append each tokenized list of words into a list of tokenized sentences.
* Use str.lower to convert strings to lowercase.
* Please use nltk.word_tokenize to split sentences into tokens.
* If you used str.split instead of nltk.word_tokenize, there are additional edge cases to handle, such as the punctuation (comma, period) that follows a word.

In [5]:
def tokenize_sentences(sentences):
    """
    Take the previously split sentences to tokenize them
    Input: List of sentences output from the first function
    Output: List of list of tokens    
    """

    # Make everything lowercase
    # tokenize each word

    tok_sent = []

    for sent in sentences:
        sent = sent.lower()
        tokenized = nltk.word_tokenize(sent)
        tok_sent.append(tokenized)

    return tok_sent



In [6]:
# Test

sentences = ["Sky is blue.", "Leaves are green.", "Roses are red."]

tok_test = tokenize_sentences(sentences)

print(tok_test)

[['sky', 'is', 'blue', '.'], ['leaves', 'are', 'green', '.'], ['roses', 'are', 'red', '.']]


## Exercise 03
Use the two functions that you have just implemented to get the tokenized data.

1. Split the data into sentences.
2. Tokenize those sentences.

In [7]:
def tokenize_data(data):
    """
    Take the data from the assignment and tokenize it
    Input: Data, a str read from the file
    Output: list of lists with tokenized data
    """

    sentences = split_to_sentences(data)
    tokens = tokenize_sentences(sentences)

    return tokens

In [8]:
# TEST
x = "Sky is blue.\nLeaves are green\nRoses are red."
tokenize_data(x)

[['sky', 'is', 'blue', '.'],
 ['leaves', 'are', 'green'],
 ['roses', 'are', 'red', '.']]

## Split into train and test sets.

In [9]:
tokenized_data = tokenize_data(data)

random.seed(13)

random.shuffle(tokenized_data)

train_size = 0.8

train_data = tokenized_data[0:int(train_size * len(tokenized_data))]
test_data = tokenized_data[int(train_size * len(tokenized_data)):]

print("The {} data is split in {} train and {} test examples".format(len(tokenized_data), len(train_data), len(test_data)))

print("The first train sample is:", train_data[0])
print("The first test sample is:", test_data[0])

The 47961 data is split in 38368 train and 9593 test examples
The first train sample is: ['i', 'miss', 'you', 'too', '!']
The first test sample is: ['haha', 'just', 'txt', 'me']


## Exercise 04
You won't use all the tokens (words) appearing in the data for training. Instead, you will use the more frequently used words.

You will focus on the words that appear at least N times in the data.
* First count how many times each word appears in the data.
* You will need a double for-loop, one for sentences and the other for tokens within a sentence.

* If you decide to import and use defaultdict, remember to cast the dictionary back to a regular 'dict' before returning it.

In [10]:
def count_words(tokenized_sentences):
    """
    Count how many times a word appears in the data
    Input:
    List of list of words
    Output:
    Dictionary where keys are words and values are the counts
    """
    word_counts = {}

    # Go through each sentence. Inside each sentence go through each word.
    for sent in tokenized_sentences:
        for w in sent:
          # If the word is not included yet, include it and count that I have seen it once.
            if w not in word_counts.keys():
                word_counts[w] = 1
          # If the word already exists, add 1.
            else:
                word_counts[w] += 1
    
    return word_counts


In [11]:
test_a = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]

a = count_words(test_a)

print(a)

{'sky': 1, 'is': 1, 'blue': 1, '.': 3, 'leaves': 1, 'are': 2, 'green': 1, 'roses': 1, 'red': 1}


## Handling 'Out of Vocabulary' words
If your model is performing autocomplete, but encounters a word that it never saw during training, it won't have an input word to help it determine the next word to suggest. The model will not be able to predict the next word because there are no counts for the current word.

* This 'new' word is called an 'unknown word', or out of vocabulary (OOV) words.
* The percentage of unknown words in the test set is called the OOV rate.

To handle unknown words during prediction, use a special token to represent all unknown words 'unk'.

* Modify the training data so that it has some 'unknown' words to train on.
* Words to convert into "unknown" words are those that do not occur very frequently in the training set.
* Create a list of the most frequent words in the training set, called the closed vocabulary .
* Convert all the other words that are not part of the closed vocabulary to the token 'unk'.

## Exercise 05
You will now create a function that takes in a text document and a threshold 'count_threshold'.

Any word whose count is greater than or equal to the threshold 'count_threshold' is kept in the closed vocabulary.
used that you want to keep, returns the document containing only the word closed vocabulary and the word unk.

In [12]:
def get_words_with_nplus_freq(tokenized_sentences, count_threshold):
    """
    Keep only the words that appear frequently. I will replace the others by <UNK>

    input: the tokenized sentences (list of lists) and a number
    Output: Vocabulary list    
    """

    closed_vocab = []
    word_count = count_words(tokenized_sentences)

    for word, count in word_count.items():
        if count >= count_threshold:
            closed_vocab.append(word)

    return closed_vocab

In [13]:
# Test

a_sentences = [['sky', 'is', 'blue', '.'],
                       ['leaves', 'are', 'green', '.'],
                       ['roses', 'are', 'red', '.']]

a_vocab = get_words_with_nplus_freq(a_sentences, count_threshold = 2)
print(a_vocab)

['.', 'are']


## Exercise 06
The words that appear 'count_threshold' times or more are in the 'closed vocabulary.

* All other words are regarded as 'unknown'.
* Replace words not in the closed vocabulary with the token "<unk>".

In [14]:
def replace_oov_unk(tokenized_sentences, vocabulary, unknown_token = "<UNK>"):
    """
    Replace words outside the vocab with <UNK>
    Input:
    Tokenized sentence, a list of lists with the words
    Vocab: a list of strings with the possible words
    Output:
    A modified list of list with the tokenized sentences
    """

    voc_set = set(vocabulary) # Faster search
    mod_sentences = []

    for sent in tokenized_sentences:
        curr_sentence = []
        for w in sent:
            if w in voc_set:
                curr_sentence.append(w)
            else:
                curr_sentence.append(unknown_token)
        mod_sentences.append(curr_sentence)

    return mod_sentences


In [15]:
tok_sent = [["dogs", "run"], ["cats", "sleep"]]

voc = ["dogs", "sleep"]

test_replace = replace_oov_unk(tok_sent, voc)
print("Original senteces:", tok_sent)
print("Modified sentences:", test_replace)

Original senteces: [['dogs', 'run'], ['cats', 'sleep']]
Modified sentences: [['dogs', '<UNK>'], ['<UNK>', 'sleep']]


## Exercise 07
Now we are ready to process our data by combining the functions that you just implemented.

* Find tokens that appear at least count_threshold times in the training data.
* Replace tokens that appear less than count_threshold times by "<unk>" both for training and test data.

In [16]:
def preprocess_data(train_data, test_data, count_threshold):
    """
    Find tokens that appear N times in the train data
    Replace the ones that don't appear enough by a <UNK> token
    Input:
    Train and test data: List of lists with the tokenized sentences
    Output:
    A tuple with train and test_data modified with the UNK replacement
    A vocabulary list
    """

    vocabulary = get_words_with_nplus_freq(train_data, count_threshold)

    train_data_replaced = replace_oov_unk(train_data, vocabulary)

    test_data_replaced = replace_oov_unk(test_data, vocabulary)

    return train_data_replaced, test_data_replaced, vocabulary

In [17]:
tmp_train = [["Sky", "is", "blue", "."], ["leaves", "are", "green", "."]]

tmp_test = [['roses', 'are', 'red', '.']]

tmp_train_rep, tmp_test_rep, tmp_voc = preprocess_data(tmp_train, tmp_test, count_threshold = 1)

print("tmp_train_repl")
print(tmp_train_rep)
print()
print("tmp_test_repl")
print(tmp_test_rep)
print()
print("tmp_vocab")
print(tmp_voc)

tmp_train_repl
[['Sky', 'is', 'blue', '.'], ['leaves', 'are', 'green', '.']]

tmp_test_repl
[['<UNK>', 'are', '<UNK>', '.']]

tmp_vocab
['Sky', 'is', 'blue', '.', 'leaves', 'are', 'green']


# Part 2: Develop n-gram based language models
In this section, you will develop the n-grams language model.

* Assume the probability of the next word depends only on the previous n-gram.
* The previous n-gram is the series of the previous 'n' words.

The conditional probability for the word at position 't' in the sentence, given that the words preceding it are $w_{t-1}, w_{t-2} \cdots w_{t-n}$ is:

$$ P(w_t | w_{t-1}\dots w_{t-n}) \tag{1}$$
You can estimate this probability by counting the occurrences of these series of words in the training data.

* The probability can be estimated as a ratio, where
* The numerator is the number of times word 't' appears after words t-1 through t-n appear in the training data.
* The denominator is the number of times word t-1 through t-n appears in the training data.
$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$

* The function $C(\cdots)$ denotes the number of occurence of the given sequence.
$\hat{P}$ means the estimation of $P$.
* Notice that denominator of the equation (2) is the number of occurence of the previous $n$ words, and the numerator is the same sequence followed by the word $w_t$.

Later, you will modify the equation (2) by adding k-smoothing, which avoids errors when any counts are zero.

The equation (2) tells us that to estimate probabilities based on n-grams, you need the counts of n-grams (for denominator) and (n+1)-grams (for numerator).

## Exercise 08
Next, you will implement a function that computes the counts of n-grams for an arbitrary number $n$.

When computing the counts for n-grams, prepare the sentence beforehand by prepending $n-1$ starting markers "<--S-->" to indicate the beginning of the sentence.

* For example, in the bi-gram model (N=2), a sequence with two start tokens ""<--S-->""<--S-->"" should predict the first word of a sentence.
* So, if the sentence is "I like food", modify it to be ""<--S-->""<--S-->" I like food".
* Also prepare the sentence for counting by appending an end token "<--E-->" so that the model can predict when to finish a sentence.

Technical note: In this implementation, you will store the counts as a dictionary.

* The key of each key-value pair in the dictionary is a tuple of n words (and not a list)
* The value in the key-value pair is the number of occurrences.
* The reason for using a tuple as a key instead of a list is because a list in Python is a mutable object (it can be changed after it is first created). A tuple is "immutable", so it cannot be altered after it is first created. This makes a tuple suitable as a data type for the key in a dictionary.
* To prepend or append, you can create lists and concatenate them using the + operator
* To create a list of a repeated value, you can follow this syntax: ['a'] * 3 to get ['a','a','a']
* To set the range for index 'i', think of this example: An n-gram where n=2 (bigram), and the sentence is length N=5 (including two start tokens and one end token). So the index positions are [0,1,2,3,4]. The largest index 'i' where a bigram can start is at position i=3, because the word tokens at position 3 and 4 will form the bigram.
* Remember that the range() function excludes the value that is used for the maximum of the range.  range(3)  produces (0,1,2) but excludes 3.

In [18]:
def count_n_grams(data, n, start_token = "<S>", end_token = "<E>"):
    """
    Count all N-grams in the data
    Input:
      training data, a list of lists of words
      n is the n in ngrams
    Output:
      A dictionary where the keys are tuples of ngrams and values are the counts
    """

    n_grams = {} # Empty dict

    for sentence in data:
        # Add start token N times and end token once
        sentence = [start_token] * n + sentence + [end_token]
        # Make it a tuple (for the dict)
        sentence = tuple(sentence)

        # Now i is the start of the sentence (indexing at 0)
        m = len(sentence) if n == 1 else len(sentence) - 1

        for i in range(m):
            n_gram = sentence[i:i+n]

            if n_gram in n_grams.keys(): # If I have it, add one to the count
                n_grams[n_gram] += 1
            else:
                n_grams[n_gram] = 1 # If I don't have it yet, add it
        
    return n_grams


    

In [19]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]

print("Unigram")
uni = count_n_grams(sentences, 1)
print(uni)
print("Bigrams")
bi = count_n_grams(sentences, 2)
print(bi)

Unigram
{('<S>',): 2, ('i',): 1, ('like',): 2, ('a',): 2, ('cat',): 2, ('<E>',): 2, ('this',): 1, ('dog',): 1, ('is',): 1}
Bigrams
{('<S>', '<S>'): 2, ('<S>', 'i'): 1, ('i', 'like'): 1, ('like', 'a'): 2, ('a', 'cat'): 2, ('cat', '<E>'): 2, ('<S>', 'this'): 1, ('this', 'dog'): 1, ('dog', 'is'): 1, ('is', 'like'): 1}


## Exercise 09
Next, estimate the probability of a word given the prior 'n' words using the n-gram counts.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n)}{C(w_{t-1}\dots w_{t-n})} \tag{2} $$
This formula doesn't work when a count of an n-gram is zero..

* Suppose we encounter an n-gram that did not occur in the training data.
* Then, the equation (2) cannot be evaluated (it becomes zero divided by zero).

A way to handle zero counts is to add k-smoothing.

* K-smoothing adds a positive constant $k$ to each numerator and $k \times |V|$ in the denominator, where $|V|$ is the number of words in the vocabulary.

$$ \hat{P}(w_t | w_{t-1}\dots w_{t-n}) = \frac{C(w_{t-1}\dots w_{t-n}, w_n) + k}{C(w_{t-1}\dots w_{t-n}) + k|V|} \tag{3} $$

For n-grams that have a zero count, the equation (3) becomes $\frac{1}{|V|}$.

* This means that any n-gram with zero count has the same probability of $\frac{1}{|V|}$.

Define a function that computes the probability estimate (3) from n-gram counts and a constant $k$.

* The function takes in a dictionary 'n_gram_counts', where the key is the n-gram and the value is the count of that n-gram.
* The function also takes another dictionary n_plus1_gram_counts, which you'll use to find the count for the previous n-gram plus the current word.
* To define a tuple containing a single value, add a comma after that value. * * For example: ('apple',) is a tuple containing a single string 'apple'
* To concatenate two tuples, use the '+' operator

In [20]:
def estimate_probability(word, previous_ngram, n_gram_counts, n_plus1_gram_counts, vocab_size, k = 1):
    """
    compute the probability of Ngrams based on eq. 3 above.
    Input:
    word is the current word
    previous_ngram: sequence of previous n words
    n_gram counts: dict with tuples of words as keys and counts as values
    n_plus1_grams counts: Same as above, with one extra word
    vocab_size: len of vocab
    Output:
    Probability, a number    
    """
    # Convert list to tuple to use as a key
    previous_n_gram = tuple(previous_ngram)

    # Calc the denominator. 
    # If the previous n_gram exists in n_gram_counts, take the value
    # Otherwise set it as 0
    
    previous_ngram_count = n_gram_counts[previous_n_gram] if previous_n_gram in n_gram_counts else 0
    
    denominator = previous_ngram_count + k * vocab_size

    # Define the nplus1_gram as the previous one plus the word

    n_plus1_gram = previous_n_gram + (word,) # Remark, it's a tuple!

    # Same procedure as before:
    n_plus1_gram_count = n_plus1_gram_counts[n_plus1_gram] if n_plus1_gram in n_plus1_gram_counts else 0

    numerator = n_plus1_gram_count + k

    prob = numerator/denominator

    return prob

In [21]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]

unique_words = list(set(sentences[0] + sentences[1]))

unigram_counts = count_n_grams(sentences, 1)

bigram_counts = count_n_grams(sentences, 2)

tmp_prob = estimate_probability("cat", "a", unigram_counts, bigram_counts, len(unique_words))

print(f"The probability of cat given a is: {tmp_prob:.4f}")

The probability of cat given a is: 0.3333


Estimate probabilities for all words

The function defined below loops over all words in vocabulary to calculate probabilities for all possible words.


In [22]:
def estimate_all_probabilities(previous_n_gram, n_gram_counts, n_plus1_counts, vocabulary, k = 1):
    """
    Estimate the probabilities of next words using the n-gram counts with k-smoothing
    
    Args:
        previous_n_gram: A sequence of words of length n
        n_gram_counts: Dictionary of counts of (n+1)-grams
        n_plus1_gram_counts: Dictionary of counts of (n+1)-grams
        vocabulary: List of words
        k: positive constant, smoothing parameter
    
    Returns:
        A dictionary mapping from next words to the probability.
    """

    previous_n_gram = tuple(previous_n_gram)

    # Add <END> and <UNK> tokens. I don't need to add <S> because I shouldn't allow to output <S>

    vocabulary = vocabulary + ["<END>", "<UNK>"]

    vocab_size = len(vocabulary)

    probabilities = {}

    for word in vocabulary:
        prob = estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_counts, vocab_size, k)

        probabilities[word] = prob

    
    return probabilities


In [23]:
sentences = [['i', 'like', 'a', 'cat'],
             ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
estimate_all_probabilities("a", unigram_counts, bigram_counts, unique_words, k=1)

{'<END>': 0.09090909090909091,
 '<UNK>': 0.09090909090909091,
 'a': 0.09090909090909091,
 'cat': 0.2727272727272727,
 'dog': 0.09090909090909091,
 'i': 0.09090909090909091,
 'is': 0.09090909090909091,
 'like': 0.09090909090909091,
 'this': 0.09090909090909091}

Count and probability matrices
As we have seen so far, the n-gram counts computed above are sufficient for computing the probabilities of the next word.

* It can be more intuitive to present them as count or probability matrices.
* The functions defined in the next cells return count or probability matrices.

In [38]:
def make_count_matrix(n_plus1_gram_counts, vocabulary):
    """
    Return the count matrix
    Add <END> and <UNK>
    Don't add <\S> because it can't be predicted as the next word by definition    
    """
    vocabulary = vocabulary + ["<END>"] + ["<UNK>"]

    # Unique n-grams
    n_grams = []

    for n_plus1_gram in n_plus1_gram_counts.keys():
        n_gram = n_plus1_gram[0:-1] # Last element of the tuple is the ,x. I don't want that.
        n_grams.append(n_gram)
    n_grams = list(set(n_grams)) # Unique

    # Now map from n_gram to row

    row_index = {n_gram:i for i, n_gram in enumerate(n_grams)}
    # Mapping word to column
    col_index = {word:j for j, word in enumerate(vocabulary)}
    
    n_row = len(n_grams)
    n_col = len(vocabulary)
    count_matrix = np.zeros((n_row, n_col))

    for n_plus1_gram, count in n_plus1_gram_counts.items():
        n_gram = n_plus1_gram[0:-1]
        word = n_plus1_gram[-1]
        if word not in vocabulary:
            continue
        i = row_index[n_gram]
        j = col_index[word]
        count_matrix[i, j] = count

    count_matrix = pd.DataFrame(count_matrix, index = n_grams, columns = vocabulary)

    return count_matrix




In [39]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]

unique_words = list(set(sentences[0] + sentences[1]))

bigram_counts = count_n_grams(sentences, 2)

print(make_count_matrix(bigram_counts, unique_words))

         cat    a   is  this    i  dog  like  <END>  <UNK>
(is,)    0.0  0.0  0.0   0.0  0.0  0.0   1.0    0.0    0.0
(like,)  0.0  2.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(i,)     0.0  0.0  0.0   0.0  0.0  0.0   1.0    0.0    0.0
(a,)     2.0  0.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(<S>,)   0.0  0.0  0.0   1.0  1.0  0.0   0.0    0.0    0.0
(cat,)   0.0  0.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(this,)  0.0  0.0  0.0   0.0  0.0  1.0   0.0    0.0    0.0
(dog,)   0.0  0.0  1.0   0.0  0.0  0.0   0.0    0.0    0.0


In [41]:
# Now try trigrams

trigram_counts = count_n_grams(sentences, 3)

print(make_count_matrix(trigram_counts, unique_words))

             cat    a   is  this    i  dog  like  <END>  <UNK>
(i, like)    0.0  1.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(this, dog)  0.0  0.0  1.0   0.0  0.0  0.0   0.0    0.0    0.0
(is, like)   0.0  1.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(<S>, i)     0.0  0.0  0.0   0.0  0.0  0.0   1.0    0.0    0.0
(like, a)    2.0  0.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(dog, is)    0.0  0.0  0.0   0.0  0.0  0.0   1.0    0.0    0.0
(a, cat)     0.0  0.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(cat,)       0.0  0.0  0.0   0.0  0.0  0.0   0.0    0.0    0.0
(<S>, this)  0.0  0.0  0.0   0.0  0.0  1.0   0.0    0.0    0.0
(<S>, <S>)   0.0  0.0  0.0   1.0  1.0  0.0   0.0    0.0    0.0


### The following function calculates the probabilities of each word given the previous n-gram, and stores this in matrix form.

In [42]:
def make_probability_matrix(n_plus1_counts, vocabulary, k):
    count_matrix = make_count_matrix(n_plus1_counts, vocabulary)
    count_matrix += k
    prob_matrix = count_matrix.div(count_matrix.sum(axis = 1), axis = 0)

    return prob_matrix



In [47]:
sentences = [['i', 'like', 'a', 'cat'],
                 ['this', 'dog', 'is', 'like', 'a', 'cat']]
unique_words = list(set(sentences[0] + sentences[1]))
bigram_counts = count_n_grams(sentences, 2)
print("bigram probabilities")
display(make_probability_matrix(bigram_counts, unique_words, k = 1))

bigram probabilities


,cat,a,is,this,i,dog,like,<END>,<UNK>
"(is,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(like,)",0.090909,0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(i,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(a,)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(<S>,)",0.090909,0.090909,0.090909,0.181818,0.181818,0.090909,0.090909,0.090909,0.090909
"(cat,)",0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
"(this,)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(dog,)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000


In [48]:
print("trigram probabilities")
trigram_counts = count_n_grams(sentences, 3)
display(make_probability_matrix(trigram_counts, unique_words, k=1))

trigram probabilities


,cat,a,is,this,i,dog,like,<END>,<UNK>
"(i, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(this, dog)",0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(is, like)",0.100000,0.200000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
"(<S>, i)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(like, a)",0.272727,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909,0.090909
"(dog, is)",0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000
"(a, cat)",0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
"(cat,)",0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111,0.111111
"(<S>, this)",0.100000,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000
"(<S>, <S>)",0.090909,0.090909,0.090909,0.181818,0.181818,0.090909,0.090909,0.090909,0.090909
